<a href="https://colab.research.google.com/github/lingelizabeth/BWSIfinal/blob/master/Decision_Tree_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install peakutils

In [0]:
from scipy.io import *
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import pandas as pd
import random
import peakutils
from sklearn import metrics

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.interpolate import *
from scipy.signal import *

import keras
from keras import optimizers
from keras.layers import *
from keras.models import *

from sklearn.tree import DecisionTreeClassifier

Using TensorFlow backend.


In [0]:
#tf.enable_eager_execution()

In [0]:
!git clone https://github.com/liua13/BWSIFinalProject
os.chdir('./BWSIFinalProject')

fatal: destination path 'BWSIFinalProject' already exists and is not an empty directory.


In [0]:
labels = pd.read_csv("labels.csv", names = ['name', 'class'])
labels['class'] = labels['class'].replace('N', 0) # non af                
labels['class'] = labels['class'].replace('A', 1) # af
labels['class'] = labels['class'].replace('O', 2) # other      
labels['class'] = labels['class'].replace('~', 3) # nois
labels = labels.drop(["name"], axis=1)
labels.head()

,class
0,0
1,0
2,0
3,1
4,1


In [0]:
df  = pd.read_csv('allrecords.csv', names = ['record name', 'signals', 'frequency', 'sample number', 'date','time','filename', 'format', 'baseline','adc', 'zeroes', 'initial value','check','block','description', 'NaNs'])
df = df.drop(['signals', 'frequency', 'format', 'baseline', 'filename', 'adc', 'zeroes', 'check', 'block', 'description', 'NaNs'], axis=1)
df['labels'] = labels['class']
df.head()

,record name,sample number,date,time,initial value,labels
0,A00001,9000,2014-05-08,05:05:15,-127,0
1,A00002,9000,2014-05-16,11:05:25,128,0
2,A00003,18000,2014-04-15,12:04:05,56,0
3,A00004,9000,2014-05-01,11:05:48,519,1
4,A00005,18000,2013-12-23,08:12:08,-188,1


In [0]:
def get_sample_data(i):
    file = "./train" + str(int(i/1000+1)) + "/" + df.loc[i, "record name"] + '.mat'
    sample = loadmat('./' + file)["val"]
    a, b = butter(5, .05, 'highpass')
    signal = filtfilt(a, b, sample)
    """ sample and file.split('/')[2] same as get_random_data() output """
    return signal
  
def get_fft_signals(sample):
    num_datapoints = len(sample['Signal'][0])
    fft_signals_x = np.arange(num_datapoints//2 + 1)/60
    fft_signals_y = np.ndarray(shape = (7, num_datapoints//2 + 1))
    for counter, signal in enumerate(np.fft.rfft(np.transpose(sample['Signal'][0]))):
        fft_signals_y[counter] = np.abs(signal)
    """ tuple: fft_signals_x (ndarray) stores fft signal frequencies in hertz; fft_signals_y (ndarray) stores amplitude for every frequency; 'FFT' stores signal type """
    return (fft_signals_x, fft_signals_y, 'FFT')
  
def get_signals(sample):
    raw_signals = get_raw_signals(sample)
    fft_signals = get_fft_signals(sample)
    """ raw_signals and fft_signals same as get_raw_signals() and get_fft_signals() output, respectively """
    return raw_signals, fft_signals

In [0]:
def calc_rr(signal):
    '''Calculates the R-R (peak-peak) data required for further analysis.
    Uses calculated measures stored in the working_data{} dict to calculate
    all required peak-peak datasets. Stores results in the working_data{} dict.
    Keyword arguments:
    sample_rate -- the sample rate of the data set
    '''
    #peaklist = np.array(working_data['peaklist'])
    peaklist = peakutils.indexes(signal[0], thres = .5)
    sample_rate = 300.
    #delete first peak if within first 150ms (signal might start mid-beat after peak)
    #if len(peaklist) > 0:
    #    if peaklist[0] <= ((sample_rate / 1000.0) * 150):
    #        peaklist = np.delete(peaklist, 0)
            #working_data['peaklist'] = peaklist
            #working_data['ybeat'] = np.delete(working_data['ybeat'], 0)

    rr_list = (np.diff(peaklist) / sample_rate) * 1000.0
    rr_diff = np.abs(np.diff(rr_list))
    rr_sqdiff = np.power(rr_diff, 2)
    return rr_diff#rr_list

def get_periodicity(signal):
  return np.var(calc_rr(signal))

def calc_RMSSD(rr_list):
  N = len(rr_list)
  sum_sq = sum([i**2 for i in rr_list])
  frac = 1/float(N-1)
  return (frac*sum_sq)**.5

In [0]:
def get_bar_fft(signal): #bar version of fft
  L = len(signal)
  fft = np.fft.rfft(signal)[:1000]
  Fs = 300
  t = np.arange(L/2 +1)/(L/Fs) 
  barx=np.linspace(0, 1000, 20) #only does the first 1000 in the fft
                                #returns 21 numbers
  bary = np.zeros(shape=barx.shape)

  for x in range(len(barx)-1):
    bary[x] = sum(abs(fft[int(barx[x]):int(barx[x+1])]))

  #bary = bary/sum(bary)
  return bary

In [0]:
def get_variance(signals, frame, interval):
    partial_array = np.ndarray(shape = (signals.shape[0], (signals.shape[1]-frame)//interval+1))
    for signal_index, signal in enumerate(signals):
        for partial_index in range((len(signal)-frame)//interval+1):
            
            """ Change feature extraction here """
            partial_array[signal_index][partial_index] = np.var(signal[(interval*partial_index):(interval*partial_index)+frame])
     
    """ partial_array (ndarray) stores multiple features for each signal"""
    return partial_array
  
def consolidate(signals):
    total_array = np.ndarray(shape = (signals.shape[0], 1))
    for signal_index, signal in enumerate(signals):
        
        """ Change feature extraction here """
        if np.sum(signal)>0:
            total_array[signal_index] = int(np.max(signal)/np.sum(signal)*100)
        else:
            total_array[signal_index] = 0
            
    """ total_array (ndarray) stores single feature for each signal """
    return total_array
  
def get_features(start, end):
    features = np.array([])
    for i in range(start, end):
        sample = get_sample_data(i)/1000.
        f1 = consolidate(get_variance(sample, 200, 200))
        f2 = get_periodicity(sample)
        f3 = get_bar_fft(np.array(sample)[0])
        feature_array = np.concatenate((f1[0].flatten(), [f2], f3.flatten()))
        features = np.append(features, feature_array)#np.hstack((f1[0][0], f2)))
    """ np.transpose(features) (ndarray) stores final features for training/testing; order (ndarray) stores order in which samples were retrieved """
    return np.transpose(features)

In [0]:
temp_data = get_features(0, 8000)
test_data = get_features(8000, 8528)

#labels = tf.keras.utils.to_categorical(labels)
temp_labels = labels[:8000]
test_labels = labels[8000:8528]

temp_shape = 8000, int(temp_data.shape[0]/8000), 1
test_shape = 528, int(test_data.shape[0]/528), 1

temp_data = temp_data.reshape(temp_shape)
test_data = test_data.reshape(test_shape)

train_data, val_data, train_labels, val_labels = train_test_split(temp_data, temp_labels, test_size = 0.25)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [0]:
train_data = train_data.reshape(6000,22)

In [0]:
val_data = val_data.astype('float32')
val_data = val_data.reshape((2000, 22))

In [0]:
print(train_data.shape, train_labels.shape)
print(val_data.shape)

(6000, 22) (6000, 1)
(2000, 22, 1)


In [0]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='gini', max_depth = 100)
for i in range(len(train_data)):
  train_data[i] = np.nan_to_num(train_data[i])     
for i in range(len(val_data)):
  val_data[i] = np.nan_to_num(val_data[i])  
clf = clf.fit(train_data, train_labels)
y_pred_dt = clf.predict(val_data)

In [0]:
from sklearn.metrics import accuracy_score
print(accuracy_score(val_labels, y_pred_dt))
print(metrics.confusion_matrix(y_pred_dt, val_labels))
print(y_pred_dt)
print(np.unique(y_pred_dt, return_counts=True))

0.516
[[776 105 279  24]
 [ 92  31  60   4]
 [288  53 201  25]
 [ 21   5  12  24]]
[0 0 2 ... 2 2 2]
(array([0, 1, 2, 3]), array([1184,  187,  567,   62]))


In [0]:
model = []
model = Sequential()

model.add(Conv1D(256, activation=tf.nn.relu, kernel_size=2, input_shape=(23, 1)))
#model.add(Conv1D(128, activation=tf.nn.relu, kernel_size=2,))
#model.add(Conv1D(64, activation=tf.nn.tanh, kernel_size=2,))

model.add(Flatten())
model.add(Dense(4, activation=tf.nn.softmax))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 22, 256)           768       
_________________________________________________________________
flatten_4 (Flatten)          (None, 5632)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 22532     
Total params: 23,300
Trainable params: 23,300
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=0.01), metrics=['accuracy'])

In [0]:
y = model.fit(train_data, train_labels, epochs=50, validation_data=(val_data, val_labels))

ValueError: ignored

In [0]:
val_pred_labels = model.predict(val_data)

In [0]:
metrics.confusion_matrix(np.argmax(val_pred_labels, axis=1), np.argmax(val_labels, axis=1))

array([[1165,  185,  580,   70],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0]])

In [0]:
test_pred_labels = model.predict(test_data)

In [0]:
metrics.confusion_matrix(np.argmax(test_pred_labels, axis=1), np.argmax(test_labels, axis=1))

array([[291,  52, 163,  22],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0]])

In [0]:
np.argmax(pred_labels, axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
pred_labels

array([[0.6025164 , 0.08281264, 0.27738595, 0.03728497],
       [0.6025164 , 0.08281264, 0.27738595, 0.03728497],
       [0.6025164 , 0.08281264, 0.27738595, 0.03728497],
       ...,
       [0.6025164 , 0.08281264, 0.27738595, 0.03728497],
       [0.6025164 , 0.08281264, 0.27738595, 0.03728497],
       [0.6025164 , 0.08281264, 0.27738595, 0.03728497]], dtype=float32)